In [1]:
import jax.numpy as jnp
from jax import random
from jax.config import config
config.update("jax_enable_x64", True)

x = random.uniform(random.PRNGKey(0), (1000,))


In [3]:
x.dtype

dtype('float64')

In [9]:
import pandas as pd

d = {'col1': [0, 1, 2, 3], 'col2': [2,3,4,5]}
d = pd.DataFrame(data=d)


In [12]:
pd.DataFrame(data=d).to_csv('haha.csv', encoding='utf-8', index=False)

In [11]:
!pwd

/Users/hudsonchen/research/fx_bayesian_quaduature/CBQ/notebooks
